In [1]:
from utils import *

data_path = "../data/XPSG - available resources/"
train_path = "../data/train/" # path of processed training data
valid_path = "../data/valid/" # path of processed validation data
processed_path = "../data/processed/"
model_path = '../models/'

train_dir = data_path+"French Ligue One 20162017 season - Match Day 1- 19/" # path of raw data
player_info = data_path+"Players and IDs - F40 - L1 20162017.xml"
test_before_change_eg = data_path+"Example test base before changes specified in the Rules -f24-24-2016-853139-eventdetails_test_hackathon_1.xml"
test_change_eg = data_path+"Example test base file - f24-24-2016-853285-eventdetails_test_hackathon_2.xml"

/Users/mhliu/Program/anaconda3/envs/ml/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
def construct_one_train(choice_xml, low, high):
    """
    Construct one training set.
    args:
        choice_xml: the xml of the chosen game whose type is lxml.etree._ElementTree.
        low: the start minute of the game.
        high: the end minute of the game.
    return:
        train_xml: the xml of one validation set whose type is lxml.etree._ElementTree.
        label_csv: the csv file of label results of this validation set.
        
    """
    suff_plyr_list = suff_plyr(choice_xml)
    if suff_plyr_list==[]:
        return [None, None]
    
    pick_plyr_id = suff_plyr_list[rnd.randint(0,len(suff_plyr_list)-1)] # randomly choose a sufficient player
    events = choice_xml.xpath('//Event')
    if low<45:
        half_type = 1
    else:
        half_type = 2
    half_events = [i for i in events if i.attrib['period_id']==str(half_type)]
    pick_events = [i for i in half_events if low<=(int(i.attrib['min'])+float(i.attrib['min'])/60)<=high]
    # check there are suff_plyrs in events
    if not suff_plyr_event(pick_events, suff_plyr_list):
        return [None, None]
    
    game = choice_xml.xpath('//Game')[0]
    home_id = game.attrib['home_team_id']
    away_id = game.attrib['away_team_id']
    
    
    game.attrib['timestamp'] = "" 
    for i in game.attrib.keys(): # substitute before info
        game.attrib[i] = ""
        
    for i in pick_events: # substitute team_id and other info
        # if 'player_id' in i.attrib:
        #    i.attrib['player_id'] = str(int(i.attrib['player_id']==pick_plyr_id))
        if 'team_id' in i.attrib:
            i.attrib['team_id'] = str(int(i.attrib['team_id']==home_id))
        if 'id' in i.attrib:
            i.attrib['id'] = ""
        if 'timestamp' in i.attrib:
            i.attrib['timestamp'] = ""
        if 'last_modified' in i.attrib:
            i.attrib['last_modified'] = ""
        if 'version' in i.attrib:
            i.attrib['version'] = ""
        
        for j in i.xpath('Q'):
            j.attrib['id'] = ""
            # When Type ID=140 and 141, qualifier_id=140/141 appears, replace the values by ""
            if ('qualifier_id' in j.attrib) and (j.attrib['qualifier_id'] == "140" or j.attrib['qualifier_id'] == "141"):
                j.attrib['qualifier_id'] = j.attrib['value'] = ""
        
    for i in pick_events[0:-10]: # substitute ball coor.
        i.attrib['x'] = i.attrib['y'] = "0.0"
    
    for i in pick_events[-10:]: # substitute the last 10 events
        i.attrib['outcome'] = ""
        for j in i.xpath('Q'):
            j.getparent().remove(j) # same as example
        #     j.attrib['qualifier_id'] = j.attrib['value'] = ""
    
    next_event_idx = choice_xml.xpath('//Event').index(pick_events[-1])+1 
    next_event = events[next_event_idx] # next event
    results = str(pick_plyr_id[1:]) + ',' + str(int(next_event.attrib['team_id']==home_id)) + ',' + \
                            str(next_event.attrib['x'])+ ',' + str(next_event.attrib['y']) # the label results
    label_csv = str(results)
    
    other_event = list(set(choice_xml.xpath('//Event')).difference(set(pick_events)))
    for j in other_event:
            j.getparent().remove(j)            
    tr_xml = choice_xml # Construct val_xml
    
    return [tr_xml, label_csv]

In [3]:
def construct_train_sets(save_path=train_path):
    """
    Construct numbers of validation set.
    args:
        save_path: the path of saving validation sets.
        val_num: the number of validation sets.
    return:
        a bool if the construction is success. 
    """
    files= os.listdir(train_dir)
    files = [i for i in files if i[0:3]=='f24']
    choice_min = [i*7.5 for i in range(11)]
    for choice_file_idx in tqdm(range(len(files))):
        for low in choice_min:
            # do while loop until returns are not None
            choice_xml = lxml.etree.parse(train_dir+files[choice_file_idx]) # randomly choose one game
            tr_xml, label_csv = construct_one_train(choice_xml, low, low+15)
            if not os.path.exists(save_path+str(choice_file_idx)):
                os.mkdir(save_path+str(choice_file_idx))
            if tr_xml==None:
                print(choice_file_idx, ' is none')
                continue
            try:
                tr_xml.write(save_path+str(choice_file_idx)+'/tr_' + str(low) + '.xml')
                with open(save_path+str(choice_file_idx)+'/label_' + str(low) + '.csv', 'w') as f:
                    f.write(label_csv)
            except IOError:
                print("Write error!")
                return False

In [4]:
def suff_plyr_event(pick_events, suff_plyr_list):
    """
    Justify if there are players in suff_plyr_list is in pick_events.
    args:
        pick_events: the picked events of the chosen game.
    return:
        Bool type.
        
    """
    for i in pick_events:
        if 'player_id' in i.attrib:
            player_id = 'p'+i.attrib['player_id']
            if player_id in suff_plyr_list:
                return True
    return False

In [127]:
construct_train_sets()









  0%|          | 0/190 [00:00<?, ?it/s]







  1%|          | 1/190 [00:01<05:13,  1.66s/it]







  1%|          | 2/190 [00:03<05:06,  1.63s/it]







  2%|▏         | 3/190 [00:05<05:13,  1.68s/it]







  2%|▏         | 4/190 [00:06<05:11,  1.67s/it]







  3%|▎         | 5/190 [00:08<05:08,  1.67s/it]







  3%|▎         | 6/190 [00:09<05:05,  1.66s/it]







  4%|▎         | 7/190 [00:11<05:01,  1.65s/it]







  4%|▍         | 8/190 [00:13<05:00,  1.65s/it]







  5%|▍         | 9/190 [00:14<04:53,  1.62s/it]







  5%|▌         | 10/190 [00:16<04:51,  1.62s/it]







  6%|▌         | 11/190 [00:18<04:48,  1.61s/it]







  6%|▋         | 12/190 [00:19<04:50,  1.63s/it]







  7%|▋         | 13/190 [00:21<04:45,  1.61s/it]







  7%|▋         | 14/190 [00:23<04:51,  1.66s/it]







  8%|▊         | 15/190 [00:24<04:54,  1.69s/it]







  8%|▊         | 16/190 [00:26<04:48,  1.66s/it]







  9%|▉         | 17/190 [00:27<04:42,  1.64s/it]







  

In [5]:
def trans_train_set_to_seq_data(path=train_path):
    """
    Transfer the training data which are in .xml and .csv into 3 sequence data (e.g. .csv).
    3 sequences are:
        - team event sequence (event sequences before the last 10)
        - event sequence (the last 10 events)
        - player sequence (all event sequence about some particular player)
    args:
        
    return:
        Bool type.
        
    """
    dirs = os.listdir(path)
    dirs = [i for i in dirs if '.' not in i]
    dirs = list(range(len(dirs)))
    for d in dirs:
        print(d)
        d = str(d)
        construct_team_seq(path+d+'/')
        construct_event_seq(path+d+'/')
        #construct_player_seq(path+d+'/')

In [13]:
def get_time_fea(df):
    """
    Get time feature of given dataframe.
    args:
        df: the given dataframe.
    return:
        df: df with time feature.
        
    """
    
    df.insert(0,'game_time', df['min']*60+df['sec'])
    df.insert(1,'time_dis_last_event', df['game_time'].shift(1))
    df['time_dis_last_event'] = df.game_time - df.time_dis_last_event
    df.loc[0, ['time_dis_last_event']] = 0
    return df

In [7]:
def get_space_fea(df):
    """
    Get space feature of given dataframe.
    args:
        df: the given dataframe.
    return:
        df: df with time feature.
        
    """
    ### space feature
    ## field
    df['field_r'] = (df['x']>50).astype('int32')
    df['field_l'] = (df['x']<=50).astype('int32')
    df.loc[df[df.ball_related==0].index, 'field_l'] = 0
    ## zone         
    df['left_zone'] = ((0<=df['y']) & (df['y']<=21.1)).astype('int32')
    df['middle_zone'] = ((21.1<df['y']) & (df['y']<78.9)).astype('int32')
    df['right_zone'] = ((78.9<=df['y']) & (df['y']<=100)).astype('int32') 
    df.loc[df[df.ball_related==0].index, 'left_zone'] = 0
    ## penal zone            
    df['penal_zone_l'] = (((0<=df['x']) & (df['x']<=17)) & ((21.1<=df['y']) & (df['y']<=78.9))).astype('int32')
    df['penal_zone_r'] = (((83<=df['x']) & (df['x']<=100)) & ((21.1<=df['y']) & (df['y']<=78.9))).astype('int32')
    df.loc[df[df.ball_related==0].index, 'penal_zone_l'] = 2
    
    df['penal_point'] = ((df['x']==88.5) & (df['y']==50)).astype('int32')
    df.insert(2,'ball_pos', np.sqrt(df['x']*df['x']+df['y']*df['y']))
    
    return df

In [8]:
def get_type_fea(df):
    """
    Get type feature of given dataframe.
    args:
        df: the given dataframe.
    return:
        df: df with type feature.
    """
    not_ball_related = [25, 27, 28, 30, 32, 68, 70, 18, 19, 40, 56, 57, 60, 65]
    game_related = [25, 27, 28, 30, 32, 68, 70]
    team_related = [18, 19, 40, 56, 57, 60, 65]
    
    both_related = [4, 5, 44]
    attack_related = [1, 2, 13, 14, 15, 16, 3, 42, 74, 6, 50, 49]
    defender_related = [10, 11, 41, 52, 53, 54, 58, 59, 7, 8, 12, 45, 51, 55]
    
    attack_pass = [1,2]
    attack_shot = [13, 14, 15]
    attack_goal = [16]
    attack_drib = [3, 42]
    attack_othe = [74, 6, 49, 50]
    defend_gk = [10, 11, 41, 52, 53, 54, 58, 59]
    
    ### ball related?
    df['ball_related'] = (df.type_id.apply(lambda x:(x not in not_ball_related))).astype(int)
    df['not_ball_related'] = (df.type_id.apply(lambda x:(x in not_ball_related))).astype(int)
    
    ## not ball related
    df['game_related'] = (df.type_id.apply(lambda x:(x in game_related))).astype(int)
    df['team_related'] = (df.type_id.apply(lambda x:(x in team_related))).astype(int)
    
    ## ball related
    df['both_related'] = (df.type_id.apply(lambda x:(x in both_related))).astype(int)
    df['attack_related'] = (df.type_id.apply(lambda x:(x in attack_related))).astype(int)
    df['defender_related'] = (df.type_id.apply(lambda x:(x in defender_related))).astype(int)
    # attack related
    df['attack_pass'] = (df.type_id.apply(lambda x:(x in attack_pass))).astype(int)
    df['attack_shot'] = (df.type_id.apply(lambda x:(x in attack_shot))).astype(int)
    df['attack_goal'] = (df.type_id.apply(lambda x:(x in attack_goal))).astype(int)
    df['attack_drib'] = (df.type_id.apply(lambda x:(x in attack_drib))).astype(int)
    df['attack_othe'] = (df.type_id.apply(lambda x:(x in attack_othe))).astype(int)
    # defend related
    df['defend_gk'] = (df.type_id.apply(lambda x:(x in defend_gk))).astype(int)
    df['defend_1'] = (df.type_id == 7).astype(int)
    df['defend_2'] = (df.type_id == 8).astype(int)
    df['defend_3'] = (df.type_id == 12).astype(int)
    df['defend_4'] = (df.type_id == 45).astype(int)
    df['defend_5'] = (df.type_id == 51).astype(int)
    df['defend_6'] = (df.type_id == 55).astype(int)
    
    df.drop(['type_id'], axis=1, inplace=True)
    
    return df

In [9]:
def construct_team_seq(path):
    """
    Construct the team event sequence.
    args:
        path: the path of given game's xml file.
    return:
        
        
    """
    files = os.listdir(path)
    xml_files = [i for i in files if 'xml' in i]
    csv_files = [i for i in files if 'csv' in i]
    xml_files.sort()
    csv_files.sort()
    
    for file_idx in tqdm(range(len(xml_files))):
        team0_df = pd.DataFrame({"min":[],
                             "sec":[],  
                             "type_id":[],
                             "q_num":[],
                             "keypass":[],
                             "nokeypass":[],
                             "assist":[],
                             "noassist":[],
                             })
        team1_df = pd.DataFrame({"min":[],
                             "sec":[],  
                             "type_id":[],
                             "q_num":[],
                             "keypass":[],
                             "nokeypass":[],
                             "assist":[],
                             "noassist":[],
                             })    
            
        xfile = xml_files[file_idx]
        cfile = csv_files[file_idx]
        choice_xml = lxml.etree.parse(path+xfile)
        events = choice_xml.xpath('//Event')[0:-10]
        team_0_events = [i for i in events if i.attrib['team_id']=='0']
        team_1_events = [i for i in events if i.attrib['team_id']=='1']
        
        for i in team_0_events:
            mins = int(i.attrib['min'])
            secs = int(i.attrib['sec'])
            type_id = int(i.attrib['type_id'])
            keypass = int('keypass' in i.attrib)
            assist = int('assist' in i.attrib)
            q_num = len(i.xpath('Q'))
            temp = pd.DataFrame({"min":[mins],
                             "sec":[secs],  
                             "type_id":[type_id],
                             "q_num":[q_num],
                             "keypass":[keypass],
                             "nokeypass":[1-keypass],
                             "assist":[assist],
                             "noassist":[1-assist],
                             })
            team0_df = pd.concat([team0_df,temp])
            
        for i in team_1_events:
            mins = int(i.attrib['min'])
            secs = int(i.attrib['sec'])
            keypass = int('keypass' in i.attrib)
            assist = int('assist' in i.attrib)
            q_num = len(i.xpath('Q'))
            temp = pd.DataFrame({"min":[mins],
                             "sec":[secs],  
                             "type_id":[type_id],
                             "q_num":[q_num],
                             "keypass":[keypass],
                             "nokeypass":[1-keypass],
                             "assist":[assist],
                             "noassist":[1-assist],
                             })
            team1_df = pd.concat([team1_df,temp])
            
        team0_df = get_time_fea(team0_df)
        team1_df = get_time_fea(team1_df)
        
        team0_df = get_type_fea(team0_df)
        team1_df = get_type_fea(team1_df)
        
        gc.collect()
        
        if not os.path.exists(path+'team_seq'):
            os.mkdir(path+'team_seq')
        
        team0_df.to_csv(path+'team_seq/'+xfile[0:-4]+'_team0.tseq', index=False)
        team1_df.to_csv(path+'team_seq/'+xfile[0:-4]+'_team1.tseq', index=False)
        

In [10]:
def construct_event_seq(path):
    """
    Construct the last 10 event sequence.
    args:
        path: the path of given game's xml file.
    return:
        
    """
    files = os.listdir(path)
    xml_files = [i for i in files if 'xml' in i]
    csv_files = [i for i in files if 'csv' in i]
    xml_files.sort()
    csv_files.sort()
    
    for file_idx in tqdm(range(len(xml_files))):
        event_df = pd.DataFrame({"min":[],
                             "sec":[],  
                             "type_id":[],
                             "q_num":[],
                             "x":[],
                             "y":[],
                             "keypass":[],
                             "nokeypass":[],
                             "assist":[],
                             "noassist":[],
                             })
        xfile = xml_files[file_idx]
        cfile = csv_files[file_idx]
        choice_xml = lxml.etree.parse(path+xfile)
        events = choice_xml.xpath('//Event')[-10:]
        
        for i in events:
            mins = int(i.attrib['min'])
            secs = int(i.attrib['sec'])
            type_id = int(i.attrib['type_id'])
            keypass = int('keypass' in i.attrib)
            assist = int('assist' in i.attrib)
            q_num = len(i.xpath('Q'))
            x = float(i.attrib['x'])/100
            y = float(i.attrib['y'])/100
            temp = pd.DataFrame({"min":[mins],
                             "sec":[secs],  
                             "type_id":[type_id],
                             "q_num":[q_num],
                             "x":[x],
                             "y":[y],
                             "keypass":[keypass],
                             "nokeypass":[1-keypass],
                             "assist":[assist],
                             "noassist":[1-assist],
                             })
            event_df = pd.concat([event_df, temp])
        
        event_df = get_time_fea(event_df)       
        event_df = get_type_fea(event_df)
        event_df = get_space_fea(event_df)
        gc.collect()
        
        if not os.path.exists(path+'event_seq'):
            os.mkdir(path+'event_seq')
        
        event_df.to_csv(path+'event_seq/'+xfile[0:-4]+'_event.eseq', index=False)

In [11]:
def construct_player_seq(path):
    """
    Construct the player event sequence.
    args:
        path: the path of given game's xml file.
    return:
        
    """
    files = os.listdir(path)
    xml_files = [i for i in files if 'xml' in i]
    csv_files = [i for i in files if 'csv' in i]
    xml_files.sort()
    csv_files.sort()
    
    for file_idx in tqdm(range(len(xml_files))):
        xfile = xml_files[file_idx]
        cfile = csv_files[file_idx]
        choice_xml = lxml.etree.parse(path+xfile)
        events = choice_xml.xpath('//Event')[0:-10]
        
         # Get all player list and changed player list
        if (os.path.exists(processed_path+"all_player_data.csv")):
            all_player_df = pd.read_csv(processed_path+"all_player_data.csv")
        else:
            all_player_df = get_player_data()
        all_player_df.join_date = all_player_df.join_date.apply(
                lambda x:pd.to_datetime(x, format="%Y-%m-%d"))
        join_date_plyr = list(
            all_player_df[all_player_df.join_date < pd.to_datetime('2017-01-01', format="%Y-%m-%d")].player_id)
        
        # Get playing time data of all players
        if (os.path.exists(processed_path+"total_play_time_data.csv")):
            total_play_time_data = pd.read_csv(processed_path+"total_play_time_data.csv")
            total_play_time_data.total_playing_time = total_play_time_data.total_playing_time.apply(
                lambda x:pd.Timedelta(x))
        else:
            total_play_time_data = get_play_time(all_player_df)
        suff_time_plyr = list(total_play_time_data[total_play_time_data.total_playing_time > pd.Timedelta(minutes=800)].player_id)
        
        players = list(set(choice_xml.xpath("//Event/@player_id")))
        players = ['p'+_ for _ in players]
        players = [_ for _ in players if (_ in suff_time_plyr) and (_ in join_date_plyr)]
        
        if len(players)==0:
            print(xfile)
            continue
        
        for p in players:
            player_df = pd.DataFrame({"min":[],
                             "sec":[],  
                             "type_id":[],
                             "q_num":[],
                             "keypass":[],
                             "nokeypass":[],
                             "assist":[],
                             "noassist":[],
                             })
            p_events = [_ for _ in events if 'player_id' in _.attrib]
            p_events = [_ for _ in p_events if ('p'+_.attrib['player_id']) == p]
            
            if len(p_events)==0:
                continue
            team_id = p_events[0].attrib['team_id']
            for i in p_events:
                mins = int(i.attrib['min'])
                secs = int(i.attrib['sec'])
                type_id = int(i.attrib['type_id'])
                keypass = int('keypass' in i.attrib)
                assist = int('assist' in i.attrib)
                q_num = len(i.xpath('Q'))
                temp = pd.DataFrame({"min":[mins],
                                 "sec":[secs],  
                                 "type_id":[type_id],
                                 "q_num":[q_num],
                                 "keypass":[keypass],
                                 "nokeypass":[1-keypass],
                                 "assist":[assist],
                                 "noassist":[1-assist],
                                 })
                player_df = pd.concat([player_df, temp])
        
            player_df = get_time_fea(player_df)       
            player_df = get_type_fea(player_df)
            gc.collect()
            player_df['player_id'] = p

            if not os.path.exists(path+'player_seq'):
                os.mkdir(path+'player_seq')

            player_df.to_csv(path+'player_seq/'+xfile[0:-4]+'_'+p+'.'+str(team_id)+'.pseq', index=False)
        

In [14]:
trans_train_set_to_seq_data()

  0%|          | 0/11 [00:00<?, ?it/s]

0


  0%|          | 0/11 [00:00<?, ?it/s]

1


  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

188


  0%|          | 0/11 [00:00<?, ?it/s]

189


100%|██████████| 11/11 [00:01<00:00,  9.86it/s]


In [108]:
team_seq_file = 'tr_'+str(0.0)+'_team0.tseq'
team_seq = pd.read_csv(train_path+'0/team_seq/'+team_seq_file).values
total_team_seq = 150

In [17]:
all_player_df = pd.read_csv(processed_path+"all_player_data.csv")

In [22]:
all_player_df.groupby('position').count().player_id

position
Defender      247
Forward       151
Goalkeeper     78
Midfielder    303
Name: player_id, dtype: int64

In [28]:
all_player_df.iloc[0,3]

'1'

In [31]:
files = os.listdir(train_path+'1/')
csv_files = [i for i in files if 'csv' in i]
label_file = [i for i in csv_files if '_'+str(0.0) in i]
label_file = pd.read_csv(train_path+'1/'+label_file[0], header=None)

In [38]:
np.array(label_file.iloc[0,2:])

array([55.2, 97.1])

In [37]:
label_file.iloc[0,2:]

2    55.2
3    97.1
Name: 0, dtype: float64

In [43]:
np.array([1]).shape

(1,)